In [3]:
# Azure Blob Storage info (public access — no SAS token needed)
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = ""  # Leave blank for public read access

# Construct WASBS path
wasbs_path = f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/{blob_relative_path}"

# Set Spark config to allow access
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token
)

print("Remote blob path:", wasbs_path)

# Step 1: Read Parquet from Azure Blob
df = spark.read.parquet(wasbs_path)

# Step 2: Add 'year' column for partitioning
from pyspark.sql.functions import year

df = df.withColumn("year", year("tpepPickupDateTime"))

# Step 3: Save to Microsoft Fabric Lakehouse as a Delta table
# Make sure your notebook is attached to a Lakehouse destination!
df.write \
  .format("delta") \
  .mode("overwrite") \
  .partitionBy("year") \
  .save("Tables/yellow_tripdata")  # <- Fabric path for Lakehouse tables

print("Data written to Lakehouse table: yellow_tripdata")


StatementMeta(, 45c55d68-ae2e-4e47-8e79-53d2c71d4922, 5, Finished, Available, Finished)

📦 Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow
✅ Data written to Lakehouse table: yellow_tripdata


In [1]:

#Day level agg table, we can compare it with monthly compact agg table
%%sql
CREATE OR REPLACE TABLE fact_vendor_daily_activity AS
SELECT
  vendorID,
  DATE_TRUNC('month', tpepPickupDateTime) AS activity_month,
  CAST(tpepPickupDateTime AS DATE) AS trip_date,
  COUNT(*) AS trip_count,
  SUM(passengerCount) AS total_passengerCount,
  SUM(tripDistance) AS total_tripDistance,
  SUM(fareAmount) AS total_fareAmount,
  SUM(extra) AS total_extra,
  SUM(mtaTax) AS total_mtaTax,
  SUM(tipAmount) AS total_tipAmount,
  SUM(tollsAmount) AS total_tollsAmount,
  SUM(totalAmount) AS total_totalAmount
FROM yellow_tripdata
GROUP BY vendorID, DATE_TRUNC('month', tpepPickupDateTime), CAST(tpepPickupDateTime AS DATE)



StatementMeta(, e9074ce9-7d8f-444e-85f3-0607c9a8c6ef, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>